# Importing libraries for the project 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
import warnings
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# EDA

In [2]:
dataset = pd.read_csv("data.csv")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64958 entries, 0 to 64957
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                64958 non-null  int64 
 1   year              64958 non-null  int64 
 2   institute_type    64958 non-null  object
 3   round_no          64958 non-null  int64 
 4   quota             64958 non-null  object
 5   pool              64958 non-null  object
 6   institute_short   64958 non-null  object
 7   program_name      64958 non-null  object
 8   program_duration  64958 non-null  object
 9   degree_short      64958 non-null  object
 10  category          64958 non-null  object
 11  opening_rank      64958 non-null  int64 
 12  closing_rank      64958 non-null  int64 
 13  is_preparatory    64958 non-null  int64 
dtypes: int64(6), object(8)
memory usage: 6.9+ MB


In [3]:
dataset.drop(["id","is_preparatory"],axis = 1)

,year,institute_type,round_no,quota,pool,institute_short,program_name,program_duration,degree_short,category,opening_rank,closing_rank
0,2016,IIT,6,AI,Gender-Neutral,IIT-Bombay,Aerospace Engineering,4 Years,B.Tech,GEN,838,1841
1,2016,IIT,6,AI,Gender-Neutral,IIT-Bombay,Aerospace Engineering,4 Years,B.Tech,OBC-NCL,408,1098
2,2016,IIT,6,AI,Gender-Neutral,IIT-Bombay,Aerospace Engineering,4 Years,B.Tech,SC,297,468
3,2016,IIT,6,AI,Gender-Neutral,IIT-Bombay,Aerospace Engineering,4 Years,B.Tech,ST,79,145
4,2016,IIT,6,AI,Gender-Neutral,IIT-Bombay,Aerospace Engineering,4 Years,B.Tech,GEN-PWD,94,94
...,...,...,...,...,...,...,...,...,...,...,...,...
64953,2021,NIT,1,JK,Female-Only,NIT-Srinagar,Electronics and Communication Engineering,4 Years,B.Tech,SC,14185,24048
64954,2021,NIT,1,JK,Gender-Neutral,NIT-Srinagar,Electronics and Communication Engineering,4 Years,B.Tech,ST,2736,4171
64955,2021,NIT,1,JK,Female-Only,NIT-Srinagar,Electronics and Communication Engineering,4 Years,B.Tech,ST,10870,10870
64956,2021,NIT,1,LA,Gender-Neutral,NIT-Srinagar,Electronics and Communication Engineering,4 Years,B.Tech,GEN,166453,265454


In [4]:
Num_features = dataset[["opening_rank","closing_rank"]]

# One hot encoding for cat_features

In [5]:
Cat_features = dataset[["year", "institute_type", "round_no", "quota", "pool", 
                        "institute_short", "program_name", "program_duration", 
                        "degree_short", "category"]]
Cat_features_encoded = pd.get_dummies(Cat_features, drop_first=True, dtype=int)
Cat_features_encoded

,year,round_no,institute_type_NIT,quota_AP,quota_GO,quota_HS,quota_JK,quota_LA,quota_OS,pool_Gender-Neutral,...,degree_short_Int Msc.,category_GEN-EWS,category_GEN-EWS-PWD,category_GEN-PWD,category_OBC-NCL,category_OBC-NCL-PWD,category_SC,category_SC-PWD,category_ST,category_ST-PWD
0,2016,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2016,6,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,2016,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
3,2016,6,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,2016,6,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64953,2021,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
64954,2021,1,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0
64955,2021,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
64956,2021,1,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


# Doing a train test split

In [6]:
from sklearn.model_selection import train_test_split
X = Cat_features_encoded
y = Num_features
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model accuracies and model selection

# Hyperparameter tuning for Decision tree

In [9]:
# Assuming you have already run the EDA and one-hot encoding steps as shown in your provided code
# ... (your EDA and one-hot encoding for Cat_features_encoded) ...

# After Cat_features_encoded is created:
model_columns = Cat_features_encoded.columns.tolist()

# Save the list of column names to a pickle file
with open('mode_columns.pkl', 'wb') as f:
    pickle.dump(model_columns, f)

print("mode_columns.pkl has been created successfully with correct columns.")

mode_columns.pkl has been created successfully with correct columns.
